In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
raw_data = pd.read_csv('../data/raw/pcori_weight_cohort_20161013.csv', 
                       encoding='latin-1', low_memory=False)
raw_data.head()

,mrn,name_last,name_first,dob,sex,race,phone_1,phone_2,address_street1,address_street2,...,weight2,weight2_date,height,height_date,bmi,bmi_date,num_encount1,num_encount2,pc_flag,pc_date
0,171892,WILKINS,ELLA,9/9/1928,2,2,6.159775e+09,615-329-8150x1923,3505 HARBORWOOD CIRCLE,NaN,...,54.93,3/17/2015,162.56,4/23/2013,20.34,6/23/2015,3.0,8.0,1,6/23/2015
1,361972,HARLEY,RANDALL,11/30/1927,1,1,6.154982e+09,NaN,3800 SAM BONEY DRIVE,"THE WINDLANDS, RM 701",...,NaN,NaN,189.23,3/7/2013,22.29,9/24/2013,NaN,3.0,1,9/24/2013
2,367615,FRAZER,JOHN,1/7/1931,1,1,6.153736e+09,NaN,110 LONG VALLEY ROAD,NaN,...,68.04,9/3/2013,170.18,9/3/2014,23.49,9/3/2014,1.0,3.0,0,NaN
3,390393,WERTHAN,BERNARD,4/18/1931,1,1,6.156650e+09,615-835-3418,4309 BEEKMAN DR,NaN,...,73.11,6/12/2015,177.80,7/22/2015,23.33,7/22/2015,45.0,36.0,1,4/4/2014
4,443465,DOUGLAS,BEVERLY,9/7/1926,1,1,6.153309e+09,931-388-4187,908 WEST 7TH ST,NaN,...,NaN,NaN,175.26,3/2/2015,33.73,3/2/2015,14.0,20.0,0,NaN


In [5]:
raw_data.shape

(417486, 25)

In [6]:
raw_data.columns

Index(['mrn', 'name_last', 'name_first', 'dob', 'sex', 'race', 'phone_1',
       'phone_2', 'address_street1', 'address_street2', 'address_city',
       'address_state', 'address_zip', 'weight1', 'weight1_date', 'weight2',
       'weight2_date', 'height', 'height_date', 'bmi', 'bmi_date',
       'num_encount1', 'num_encount2', 'pc_flag', 'pc_date'],
      dtype='object')

In [7]:
raw_data.mrn.unique().shape

(417486,)

In [11]:
raw_data.patient_id.str.lstrip('0').unique().shape

(15436,)

In [13]:
raw_data.patient_id_vandy.unique().shape

(7393,)

In [7]:
raw_data.patient_id_greenway.unique().shape

(3,)

In [8]:
raw_data.patient_id_jackson.unique().shape

(6005,)

In [15]:
raw_data.participantid.unique().shape

(5312,)

In [5]:
column_names = raw_data.columns.tolist()

In [14]:
raw_data.columns[raw_data.columns.str.endswith('id')]

Index(['patient_id', 'site_id', 'jacksonid', 'participantid', 'Bid', 'Aid',
       'study_id', 'greenwayid'],
      dtype='object')

In [5]:
data_by_patient = raw_data.groupby('patient_id')

In [11]:
dd.compute(raw_data.weight.min(), raw_data.weight.max())

/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/dask/async.py:263: DtypeWarning: Columns (0,6,7,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


ValueError: could not convert string to float: '03/01/2013'

Traceback
---------
  File "/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/dask/async.py", line 263, in execute_task
    result = _execute_task(task, data)
  File "/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/dask/async.py", line 245, in _execute_task
    return func(*args2)
  File "/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/dask/compatibility.py", line 47, in apply
    return func(*args, **kwargs)
  File "/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/dask/dataframe/csv.py", line 49, in bytes_read_csv
    coerce_dtypes(df, dtypes)
  File "/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/dask/dataframe/csv.py", line 74, in coerce_dtypes
    df[c] = df[c].astype(dtypes[c])
  File "/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/pandas/core/generic.py", line 3054, in astype
    raise_on_error=raise_on_error, **kwargs)
  File "/Use

/Users/fonnescj/anaconda3/envs/dev/lib/python3.5/site-packages/dask/async.py:263: DtypeWarning: Columns (21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)
